In [1]:
import os
import pandas
import numpy as np
import nibabel as nib
from glob import glob
from nilearn import image, input_data, datasets


baseDir = os.path.join("/project", "ctb-villens", "dataset", "camcan", "derivatives",
                       "camcan_vbm", "dartel_spm12_647sub")
dataDir = os.path.join("/scratch", "cbedetti", "pac2019", "data", "nilearn")

In [2]:
# find GM images
gms = sorted(glob(os.path.join(baseDir, "mwc1", "*nii.gz")))

In [3]:
# Initiate atlases
basc = datasets.atlas.fetch_atlas_basc_multiscale_2015(data_dir=dataDir)

atlases = {}
for scale in basc.__dir__():
    if 'scale' in scale:
        atlases.update({scale: getattr(basc,scale)})

print(atlases)

{'scale007': '/scratch/cbedetti/pac2019/data/nilearn/basc_multiscale_2015/template_cambridge_basc_multiscale_nii_sym/template_cambridge_basc_multiscale_sym_scale007.nii.gz', 'scale064': '/scratch/cbedetti/pac2019/data/nilearn/basc_multiscale_2015/template_cambridge_basc_multiscale_nii_sym/template_cambridge_basc_multiscale_sym_scale064.nii.gz', 'scale012': '/scratch/cbedetti/pac2019/data/nilearn/basc_multiscale_2015/template_cambridge_basc_multiscale_nii_sym/template_cambridge_basc_multiscale_sym_scale012.nii.gz', 'scale197': '/scratch/cbedetti/pac2019/data/nilearn/basc_multiscale_2015/template_cambridge_basc_multiscale_nii_sym/template_cambridge_basc_multiscale_sym_scale197.nii.gz', 'scale122': '/scratch/cbedetti/pac2019/data/nilearn/basc_multiscale_2015/template_cambridge_basc_multiscale_nii_sym/template_cambridge_basc_multiscale_sym_scale122.nii.gz', 'scale444': '/scratch/cbedetti/pac2019/data/nilearn/basc_multiscale_2015/template_cambridge_basc_multiscale_nii_sym/template_cambridge

In [4]:
# get feature names
feature_names = []
for scale, atlas in atlases.items():
    unique_labels = np.unique(nib.load(atlas).get_data())
    feature_names += ["{}_{}".format(scale,int(i)) for i in unique_labels[1:]] # don't grab background
print('{} features in total'.format(len(feature_names)))

1227 features in total


In [5]:
# build results dataframe
subs = [_.split("-")[2] for _ in gms]
features = pandas.DataFrame(index = subs, columns = feature_names)
features.head()

,scale007_1,scale007_2,scale007_3,scale007_4,scale007_5,scale007_6,scale007_7,scale064_1,scale064_2,scale064_3,...,scale325_316,scale325_317,scale325_318,scale325_319,scale325_320,scale325_321,scale325_322,scale325_323,scale325_324,scale325_325
CC110033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC110037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC110045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC110056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC110062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# find batch size
print(len(gms))

for i in range(10,200):
    if 646%i == 0:
        print('A batch size of {} would result in {} batches'.format(i,646/i))

647
A batch size of 17 would result in 38.0 batches
A batch size of 19 would result in 34.0 batches
A batch size of 34 would result in 19.0 batches
A batch size of 38 would result in 17.0 batches


In [9]:
## Extract data

n_batches = 10

for scale, atlas in atlases.items():
    print('working on',scale)
    mskr = input_data.NiftiLabelsMasker(atlas)
    cols = [x for x in features.columns if scale in x]
    
    for i, batch in enumerate(np.array_split(gms, n_batches)):
        print('working on batch {} of {}'.format(i+1,n_batches))
        sids = [_.split("-")[2] for _ in batch]
        if not all([x in features.index for x in sids]):
            print(['>']*10,'missing subject in batch',i,['<']*10)
            continue
        imgs = image.load_img(batch)
        data = mskr.fit_transform(imgs)
        features.loc[sids, cols] = data
        features.to_csv('BASC_Features_GMDerivatives_CamCan.csv')

working on scale007
working on batch 1 of 10
working on batch 2 of 10
working on batch 3 of 10
working on batch 4 of 10
working on batch 5 of 10
working on batch 6 of 10
working on batch 7 of 10
working on batch 8 of 10
working on batch 9 of 10
working on batch 10 of 10
working on scale064
working on batch 1 of 10
working on batch 2 of 10
working on batch 3 of 10
working on batch 4 of 10
working on batch 5 of 10
working on batch 6 of 10
working on batch 7 of 10
working on batch 8 of 10
working on batch 9 of 10
working on batch 10 of 10
working on scale012
working on batch 1 of 10
working on batch 2 of 10
working on batch 3 of 10
working on batch 4 of 10
working on batch 5 of 10
working on batch 6 of 10
working on batch 7 of 10
working on batch 8 of 10
working on batch 9 of 10
working on batch 10 of 10
working on scale197
working on batch 1 of 10
working on batch 2 of 10
working on batch 3 of 10
working on batch 4 of 10
working on batch 5 of 10
working on batch 6 of 10
working on batch 

In [10]:
np.savez_compressed("BASC_Features_GMDerivatives_CamCan", a=features)